In [14]:
import kagglehub
import shutil

In [18]:
import kagglehub

path = kagglehub.dataset_download("muratkokludataset/rice-image-dataset")
print("Dataset downloaded to:", path)


Dataset downloaded to: C:\Users\felip\.cache\kagglehub\datasets\muratkokludataset\rice-image-dataset\versions\1


In [19]:
dst = r"C:\Users\felip\OneDrive - Rice University\CNN"
shutil.copytree(path, dst, dirs_exist_ok=True)

'C:\\Users\\felip\\OneDrive - Rice University\\CNN'

In [20]:
import os

data_dir = r"C:\Users\felip\OneDrive - Rice University\CNN\Rice_Image_Dataset"
classes = [d.name for d in os.scandir(data_dir) if d.is_dir()]
for cls in classes:
    print(cls, ":", len(os.listdir(os.path.join(data_dir, cls))), "images")


Arborio : 15000 images
Basmati : 15000 images
Ipsala : 15000 images
Jasmine : 15000 images
Karacadag : 15000 images


In [21]:
import os

data_dir = r"C:\Users\felip\OneDrive - Rice University\CNN\Rice_Image_Dataset"

for cls in [d.name for d in os.scandir(data_dir) if d.is_dir()]:
    subdirs = [d.name for d in os.scandir(os.path.join(data_dir, cls)) if d.is_dir()]
    if subdirs:
        print(f"⚠️ {cls} contains subfolders:", subdirs)
    else:
        print(f"✅ {cls} is clean (no nested folders)")


✅ Arborio is clean (no nested folders)
✅ Basmati is clean (no nested folders)
✅ Ipsala is clean (no nested folders)
✅ Jasmine is clean (no nested folders)
✅ Karacadag is clean (no nested folders)


In [22]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_gen = datagen.flow_from_directory(
    data_dir,
    target_size=(128, 128),
    batch_size=32,
    subset='training'
)

val_gen = datagen.flow_from_directory(
    data_dir,
    target_size=(128, 128),
    batch_size=32,
    subset='validation'
)


Found 60000 images belonging to 5 classes.
Found 15000 images belonging to 5 classes.


In [23]:
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(128,128,3)),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(5, activation='softmax')
])

model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,305,285 (12.61 MB)

 Trainable params: 3,305,285 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

In [24]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [25]:
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10
)

Epoch 1/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 1410s 751ms/step - accuracy: 0.9341 - loss: 0.1869 - val_accuracy: 0.9619 - val_loss: 0.1031
Epoch 2/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 277s 148ms/step - accuracy: 0.9755 - loss: 0.0699 - val_accuracy: 0.9909 - val_loss: 0.0267
Epoch 3/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 276s 147ms/step - accuracy: 0.9884 - loss: 0.0338 - val_accuracy: 0.9889 - val_loss: 0.0318
Epoch 4/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 277s 148ms/step - accuracy: 0.9937 - loss: 0.0188 - val_accuracy: 0.9941 - val_loss: 0.0192
Epoch 5/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 282s 150ms/step - accuracy: 0.9944 - loss: 0.0153 - val_accuracy: 0.9931 - val_loss: 0.0206
Epoch 6/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 275s 147ms/step - accuracy: 0.9958 - loss: 0.0117 - val_accuracy: 0.9965 - val_loss: 0.0131
Epoch 7/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 269s 144ms/step - accuracy: 0.9960 - loss: 0.0113 - val_accuracy: 0.9941 - val_loss: 0.0196
Epoch 8/10
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 269s 143ms/step - a